In [1]:
import os
import sys

sys.path.append("../../../../")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import copy
import torch
from datetime import datetime
from utils.helper import ModelConfig, color_print
from utils.dataset_utils.load_dataset import (
    load_data,
)
from utils.model_utils.load_model import load_model
from utils.model_utils.save_module import save_module
from utils.model_utils.evaluate import evaluate_model, get_sparsity, similar
from utils.dataset_utils.sampling import SamplingDataset
from utils.prune_utils.prune import (
    prune_concern_identification,
    recover_tangling_identification,
)

/home/jieungkim/anaconda3/envs/DecomposeTransformer/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
name = "YahooAnswersTopics"
device = torch.device("cuda:0")
checkpoint = None
batch_size = 16
num_workers = 4
num_samples = 16
ci_ratio = 0.4
ti_ratio = 0.1
seed=44
include_layers = ["attention", "intermediate", "output"]
exclude_layers = None

In [4]:
model_config = ModelConfig(name, device)
num_labels = model_config.config["num_labels"]
model, tokenizer, checkpoint = load_model(model_config)

Loading the model.

{'model_name': 'fabriceyhc/bert-base-uncased-yahoo_answers_topics', 'task_type': 'classification', 'architectures': 'bert', 'dataset_name': 'YahooAnswersTopics', 'num_labels': 10, 'cache_dir': 'Models'}

The model fabriceyhc/bert-base-uncased-yahoo_answers_topics is loaded.

In [5]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
    name, batch_size=batch_size, num_workers=num_workers, do_cache=True, seed=seed
)

{'dataset_name': 'YahooAnswersTopics', 'path': 'yahoo_answers_topics', 'config_name': 'yahoo_answers_topics', 'text_column': 'question_title', 'label_column': 'topic', 'cache_dir': 'Datasets/Yahoo', 'task_type': 'classification'}

Generating train split:   0%|          | 0/1400000 [00:00<?, ? examples/s]

Generating train split:   3%|▎         | 39000/1400000 [00:00<00:03, 376529.24 examples/s]

Generating train split:   6%|▌         | 85000/1400000 [00:00<00:03, 420007.87 examples/s]

Generating train split:  10%|▉         | 134000/1400000 [00:00<00:02, 445665.37 examples/s]

...

Tokenizing dataset:  97%|#########6| 28992/30000 [00:05<00:00, 4919.77it/s]

Tokenizing dataset:  98%|#########8| 29502/30000 [00:05<00:00, 4970.39it/s]

Tokenizing dataset: 100%|##########| 30000/30000 [00:05<00:00, 5075.35it/s]

Caching is completed.

The dataset YahooAnswersTopics is loaded

In [6]:
# print("Evaluate the original model")
# result = evaluate_model(model, model_config, test_dataloader)

In [7]:
for concern in range(num_labels):
    train = copy.deepcopy(train_dataloader)
    valid = copy.deepcopy(valid_dataloader)
    positive_samples = SamplingDataset(
        train, concern, num_samples // 2, num_labels, True, 4, device=device, resample=False, seed=seed
    )
    negative_samples = SamplingDataset(
        train, concern, num_samples // 2, num_labels, False, 4, device=device, resample=False, seed=seed
    )
    all_samples = SamplingDataset(
        train, 200, num_samples // 2, num_labels, False, 4, device=device, resample=False, seed=seed
    )
    module = copy.deepcopy(model)

    prune_concern_identification(
        module,
        model_config,
        positive_samples,
        negative_samples,
        include_layers=include_layers,
        exclude_layers=exclude_layers,
        sparsity_ratio=ci_ratio,
    )

    # print(f"Evaluate the pruned model {concern}")
    # result = evaluate_model(module, model_config, test_dataloader)
    # similar(model, module, valid, concern, num_samples, num_labels, device=device, seed=seed)

    recover_tangling_identification(
        model,
        module,
        model_config,
        positive_samples,
        negative_samples,
        recovery_ratio=ti_ratio,
        include_layers=include_layers,
        exclude_layers=exclude_layers
    )

    print(f"Evaluate the pruned model {concern}")
    result = evaluate_model(module, model_config, test_dataloader)
    similar(model, module, valid, concern, num_samples, num_labels, device=device, seed=seed)

    # save_module(module, "Modules/", f"citi_{name}_{ci_ratio-ti_ratio}p.pt")

Evaluate the pruned model 0

Evaluating:   0%|          | 0/1875 [00:00<?, ?it/s]

Evaluating:   0%|          | 1/1875 [00:00<08:53,  3.51it/s]

Evaluating:   0%|          | 4/1875 [00:00<02:51, 10.88it/s]

Evaluating:   0%|          | 7/1875 [00:00<02:05, 14.90it/s]

...

CCA coefficients mean non-concern: (0.8369474497272718, 0.8369474497272718)

Linear CKA concern: 0.9757963856323834

Linear CKA non-concern: 0.9491639787279608

Kernel CKA concern: 0.9580065068801981

Kernel CKA non-concern: 0.9264390200649371